In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import os
from typing import List

import polars as pl
from freezegun import freeze_time

from src.activities import (
    get_activities_df,
    get_day_of_week_summaries,
    get_weekly_summaries,
)
from src.types.training_week import TrainingWeekWithCoaching, TrainingSession
from src.auth_manager import get_strava_client
from src.email_manager import send_email, training_week_to_html
from src.supabase_client import upsert_training_week_with_coaching, get_training_week_with_coaching
from src.training_week_generation.training_week import generate_training_week

# Sunday Night Training Week Message

In [62]:
@freeze_time("2024-08-11 20:00:00")
def get_activities_df_wrapper(strava_client):
    return get_activities_df(strava_client)

client_preferences = "A) Training for a marathon B) This will be my second marathon C) Prefer workouts on Wednesdays and long runs on Saturdays"

# activities setup
athlete_id = os.environ["JAMIES_ATHLETE_ID"]
strava_client = get_strava_client(athlete_id)
activities_df = get_activities_df_wrapper(strava_client)

# gen training week pipeline
day_of_week_summaries = get_day_of_week_summaries(activities_df)
weekly_summaries = get_weekly_summaries(activities_df)
training_week_with_coaching = generate_training_week(
    client_preferences=client_preferences,
    weekly_summaries=weekly_summaries,
    day_of_week_summaries=day_of_week_summaries,
)

# save data to db and trigger email
upsert_training_week_with_coaching(
    athlete_id=athlete_id, training_week_with_coaching=training_week_with_coaching
)
send_email(
    subject="Training Schedule Just Dropped 🏃",
    html_content=training_week_to_html(training_week_with_coaching),
)

athlete_id='98390356' token still valid until 2024-08-17 05:16:28+00:00


{'message_id': '<202408162339.82734117080@smtp-relay.mailin.fr>',
 'message_ids': None}

# Mid-Week Update

In [67]:
import datetime

@freeze_time("2024-08-13 20:00:00")
def mock_get_activities_df(strava_client, num_weeks=8):
    return get_activities_df(strava_client, num_weeks)

@freeze_time("2024-08-13 20:00:00")
def mock_time_now():
    return datetime.datetime.now()

def get_day_of_week():
    return mock_time_now().strftime("%A")

def get_weekday_num():
    return mock_time_now().weekday()


def get_concise_activities(strava_client, num_weeks=8) -> pl.DataFrame:
    df = mock_get_activities_df(strava_client, num_weeks)
    return df.with_columns(
        pl.col('start_date_local').apply(lambda x: x.strftime("%A, %B %d, %Y %I:%M %p")),
        pl.col('distance_in_miles').apply(lambda x: round(x, 2)),
        pl.col('elevation_gain_in_feet').apply(lambda x: round(x, 2)),
        pl.col('pace_minutes_per_mile').apply(lambda x: round(x, 2)),
    ).drop('id', 'name', 'day_of_week', 'week_of_year', 'year', 'moving_time_in_minutes')


def get_mid_week_delta(
    training_week_with_coaching: TrainingWeekWithCoaching,
    activities: pl.DataFrame
) -> float:
    """
    Assumes that activities is truncated to the current week, calculating the 
    difference between target mileage and actual mileage thus far this week

    :param training_week_with_coaching: instance TrainingWeekWithCoaching
    :param activities: DataFrame of activities thus far this week
    :return: The mileage delta as a float
    """
    actual_mileage = activities['distance_in_miles'].sum()
    weekly_target = sum(sess.distance for sess in training_week_with_coaching.training_week)
    mileage_delta = weekly_target - actual_mileage

    return mileage_delta

In [68]:

athlete_id = os.environ["JAMIES_ATHLETE_ID"]
strava_client = get_strava_client(athlete_id)
training_week_with_coaching = get_training_week_with_coaching(athlete_id)
activities = get_concise_activities(strava_client, num_weeks=1)

athlete_id='98390356' token still valid until 2024-08-17 05:16:28+00:00


In [69]:
activities

start_date_local,distance_in_miles,elevation_gain_in_feet,pace_minutes_per_mile
str,f64,f64,f64
"""Monday, August 12, 2024 06:06 …",8.01,183.73,9.43
"""Tuesday, August 13, 2024 06:15…",4.01,52.49,10.64
